In [1]:
import numpy as np
import torch
from model import bertclassify, text_decoder

my_model = bertclassify(1)
my_model=my_model.cuda()

my_models=[text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda()]
for index,onemodel in enumerate(my_models):
    onemodel.load_state_dict(torch.load("model/decoder_%d.pth"%index))
my_model.load_state_dict(torch.load("model/encoder.pth"))


<All keys matched successfully>

In [3]:
import numpy as np
def myeval(pred, tar):
    pred_zero = torch.zeros_like(pred)
    pred_zero[pred > 0.5] = 1
    judge_n=pred_zero.cuda()==tar.cuda()
    error_num=0
    for index,i in enumerate(judge_n):
        if False == i[0] and tar[index,0]==1.0:
            error_num+=1
    return error_num,len(judge_n),1-error_num/len(judge_n)

from torch.utils.data import DataLoader
from dataset import TextDataset
numClass=8
train_dataset = TextDataset(True)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
allRightnum=[]
allknum=[]
for i in range(numClass):
    allknum.append(0)
    allRightnum.append(0)
allknum=np.asarray(allknum)
allRightnum=np.asarray(allRightnum)
for input_data in train_data_loader:
    de_onemodel = my_model(input_data)

    for idclass in range(numClass):
        my_models[idclass].eval()
        output = my_models[idclass](de_onemodel)
        tar=torch.unsqueeze(input_data['label'][:,idclass].cuda(),1)
        errornum,knum,acc = myeval(output, tar)
        allRightnum[idclass]+=sum(tar)-errornum
        allknum[idclass]+=sum(tar)
        print("%drecall:"%idclass,(sum(tar)-errornum)/sum(tar))
    print("allrecall",allRightnum/allknum)
    print("allnum:",allknum," allrightnum:",allRightnum)

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DA81EF1820>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 64261245-0bf6-454a-8a97-ac58c072296c)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json
10/29/2023 22:54:44 - WARNING - huggingface_hub.utils._http -   '(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DA81EF1820>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 64261245-0bf6-454a-8a97-ac58c072296c)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve


maxlen: 162
0recall: tensor([0.9333], device='cuda:0')
1recall: tensor([1.], device='cuda:0')
2recall: tensor([1.], device='cuda:0')
3recall: tensor([0.7500], device='cuda:0')
4recall: tensor([0.6667], device='cuda:0')
5recall: tensor([0.], device='cuda:0')
6recall: tensor([0.6667], device='cuda:0')
7recall: tensor([0.], device='cuda:0')
allrecall [0.93333333 1.         1.         0.75       0.66666667 0.
 0.66666667 0.        ]
allnum: [15 11  7  8  9  3  3  4]  allrightnum: [14 11  7  6  6  0  2  0]
0recall: tensor([0.9000], device='cuda:0')
1recall: tensor([1.], device='cuda:0')
2recall: tensor([0.9000], device='cuda:0')
3recall: tensor([0.9091], device='cuda:0')
4recall: tensor([0.8333], device='cuda:0')
5recall: tensor([0.], device='cuda:0')
6recall: tensor([nan], device='cuda:0')
7recall: tensor([0.], device='cuda:0')
allrecall [0.92       1.         0.94117647 0.84210526 0.73333333 0.
 0.66666667 0.        ]
allnum: [25 16 17 19 15  7  3  7]  allrightnum: [23 16 16 16 11  0  2 


KeyboardInterrupt

